In [1]:
import pandas as pd
import json
import base64
import numpy as np
from frozendict import frozendict
import glob
import requests

base_url = 'http://127.0.0.1:8000/'

# Create genres

In [2]:
data = pd.read_csv('movies_metadata.csv')
data = data.loc[np.arange(0, 500)]
data.head()

C:\Users\stepan.samsonov\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [56]:
film_ids = list(data['id'])

In [57]:
names = set()

for line in data['genres'].iteritems():
    line = eval(line[1])
    for elem in line:
        names.add(elem['name'])

In [58]:
names

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

In [59]:
descriptions = frozendict({
    'Music':
"""Musical film is a film genre in which songs sung by the characters are interwoven into the narrative, sometimes accompanied by dancing.

The songs usually advance the plot or develop the film's characters, but in some cases, they serve merely as breaks in the storyline, often as elaborate "production numbers."

The musical film was a natural development of the stage musical after the emergence of sound film technology. Typically, the biggest difference between film and stage musicals is the use of lavish background scenery and locations that would be impractical in a theater. Musical films characteristically contain elements reminiscent of theater; performers often treat their song and dance numbers as if a live audience were watching. In a sense, the viewer becomes the diegetic audience, as the performer looks directly into the camera and performs to it.
""",
    'Romance':
"""Romance films or romance movies are romantic love stories recorded in visual media for broadcast in theaters and on TV that focus on passion, emotion, and the affectionate romantic involvement of the main characters and the journey that their genuinely strong, true and pure romantic love takes them through dating, courtship or marriage. Romance films make the romantic love story or the search for strong and pure love and romance the main plot focus. Occasionally, romance lovers face obstacles such as finances, physical illness, various forms of discrimination, psychological restraints or family that threaten to break their union of love. As in all quite strong, deep, and close romantic relationships, tensions of day-to-day life, temptations (of infidelity), and differences in compatibility enter into the plots of romantic films.[1]

Romantic films often explore the essential themes of love at first sight, young with older love, unrequited romantic love, obsessive love, sentimental love, spiritual love, forbidden love/romance, platonic love, sexual and passionate love, sacrificial love, explosive and destructive love, and tragic love. Romantic films serve as great escapes and fantasies for viewers, especially if the two people finally overcome their difficulties, declare their love, and experience life "happily ever after", implied by a reunion and final kiss. In romantic television series, the development of such romantic relationships may play out over many episodes, and different characters may become intertwined in different romantic arcs.
""",
    'Family':
"""A children's film, or family film, is a film genre that contains children or relates to them in the context of home and family. Children's films are made specifically for children and not necessarily for the general audience, while family films are made for a wider appeal with a general audience in mind. Children's films come in several major forms like realism, fantasy, adventure, animation, war, musicals, and literary adaptations.
""",
    'War':
"""War film is a film genre concerned with warfare, typically about naval, air, or land battles, with combat scenes central to the drama. It has been strongly associated with the 20th century.[1][2] The fateful nature of battle scenes means that war films often end with them. Themes explored include combat, survival and escape, camaraderie between soldiers, sacrifice, the futility and inhumanity of battle, the effects of war on society, and the moral and human issues raised by war. War films are often categorized by their milieu, such as the Korean War; the most popular subject is the Second World War. The stories told may be fiction, historical drama, or biographical. Critics have noted similarities between the Western and the war film.

Nations such as China, Indonesia, Japan, and Russia have their own traditions of war film, centred on their own revolutionary wars but taking varied forms, from action and historical drama to wartime romance.

Subgenres, not necessarily distinct, include anti-war, comedy, animated, propaganda, and documentary. There are similarly subgenres of the war film in specific theatres such as the western desert (North Africa), the Pacific in the Second World War, or Vietnam; and films set in specific domains of war, such as the infantry, the air, at sea, in submarines, or at prisoner of war camps.
""",
    'Adventure':
"""Adventure films are a genre of film that typically use their action scenes to display and explore exotic locations in an energetic way.
Subgenres of adventure films include swashbuckler films, survival films, superhero films and pirate films. Main plot elements include quests for lost continents and exotic setting; the creation of empires, characters embarking on treasure and heroic journeys, travels, explorations, quests and searches for the unknown usually also having to overcome an adversary. Adventure films are often set in a period background and may include adapted stories of historical or fictional adventure heroes within the historical context. Kings, battles, rebellion or piracy are commonly seen. Adventure films may also be combined with other movie genres such as action, animation, comedy, drama, fantasy, science fiction, family, horror or war.
""",
    'Fantasy':
"""Fantasy films are films that belong to the fantasy genre with fantastic themes, usually magic, supernatural events, mythology, folklore, or exotic fantasy worlds. The genre is considered a form of speculative fiction alongside science fiction films and horror films, although the genres do overlap. Fantasy films often have an element of magic, myth, wonder, escapism, and the extraordinary.
""",
    'Animation':
"""
A cartoon is a type of illustration, possibly animated, typically in a non-realistic or semi-realistic style. The specific meaning has evolved over time, but the modern usage usually refers to either: an image or series of images intended for satire, caricature, or humor; or a motion picture that relies on a sequence of illustrations for its animation. Someone who creates cartoons in the first sense is called a cartoonist, and in the second sense they are usually called an animator.

The concept originated in the Middle Ages, and first described a preparatory drawing for a piece of art, such as a painting, fresco, tapestry, or stained glass window. In the 19th century, beginning in Punch magazine in 1843, cartoon came to refer – ironically at first – to humorous illustrations in magazines and newspapers. In the early 20th century, it began to refer to animated films which resembled print cartoons.
""",
    'Drama':
"""In film and television, drama is a genre of narrative fiction (or semi-fiction) intended to be more serious than humorous in tone. Drama of this kind is usually qualified with additional terms that specify its particular subgenre, such as "police crime drama", "political drama", "legal drama", "historical period drama", "domestic drama", or "comedy-drama". These terms tend to indicate a particular setting or subject-matter, or else they qualify the otherwise serious tone of a drama with elements that encourage a broader range of moods.

All forms of cinema or television that involve fictional stories are forms of drama in the broader sense if their storytelling is achieved by means of actors who represent (mimesis) characters. In this broader sense, drama is a mode distinct from novels, short stories, and narrative poetry or songs. In the modern era before the birth of cinema or television, "drama" within theatre was a type of play that was neither a comedy nor a tragedy. It is this narrower sense that the film and television industries, along with film studies, adopted. "Radio drama" has been used in both senses—originally transmitted in a live performance, it has also been used to describe the more high-brow and serious end of the dramatic output of radio.
""",
    'Horror':
"""A horror film is a film that seeks to elicit fear. Initially inspired by literature from authors like Edgar Allan Poe, Bram Stoker, and Mary Shelley, horror has existed as a film genre for more than a century. The macabre and the supernatural are frequent themes. Horror may also overlap with the fantasy, supernatural fiction, and thriller genres.

Horror films often aim to evoke viewers' nightmares, fears, revulsions and terror of the unknown. Plots within the horror genre often involve the intrusion of an evil force, event, or personage into the everyday world. Prevalent elements include ghosts, extraterrestrials, vampires, werewolves, demons, Satanism, evil clowns, gore, torture, vicious animals, evil witches, monsters, zombies, cannibalism, psychopaths, natural or man-made disasters, and serial killers.

Some sub-genres of horror film include low-budget horror, action horror, comedy horror, body horror, disaster horror, found footage, holiday horror, horror drama, psychological horror, science fiction horror, slasher, supernatural horror, gothic horror, natural horror, zombie horror, first-person horror, and teen horror.
""",
    'Action':
"""Action film is a film genre in which the protagonist or protagonists are thrust into a series of challenges that typically include violence, extended fighting, physical feats, and frantic chases. Action films tend to feature a resourceful hero struggling against incredible odds, which include life-threatening situations, a villain, or a pursuit which generally concludes in victory for the hero (though a small number of films in this genre have ended in victory for the villain instead). Advancements in CGI have made it cheaper and easier to create action sequences and other visual effects that required the efforts of professional stunt crews in the past. However, reactions to action films containing significant amounts of CGI have been mixed, as films that use computer animations to create unrealistic, highly unbelievable events are often met with criticism. While action has long been a recurring component in films, the "action film" genre began to develop in the 1970s along with the increase of stunts and special effects. Common action scenes in films are generally, but not limited to, car chases, fighting and gunplay or shootouts.

This genre is closely associated with the thriller and adventure genres, and they may also contain elements of spy fiction.
""",
    'Comedy':
"""Comedy is a genre of film in which the main emphasis is on humour. These films are designed to make the audience laugh through amusement and most often work by exaggerating characteristics for humorous effect. Films in this style traditionally have a happy ending (black comedy being an exception). One of the oldest genres in film, some of the very first silent movies were comedies, as slapstick comedy often relies on visual depictions, without requiring sound. When sound films became more prevalent during the 1920s, comedy films took another swing, as laughter could result from burlesque situations but also dialogue.

Comedy, compared with other film genres, puts much more focus on individual stars, with many former stand-up comics transitioning to the film industry due to their popularity. While many comic films are lighthearted stories with no intent other than to amuse, others contain political or social commentary (such as The King of Comedy and Wag the Dog).
""",
    'History':
"""Historical fiction is a literary genre in which the plot takes place in a setting located in the past. Although the term is commonly used as a synonym for the historical novel, it can also be applied to other types of narrative, including theatre, opera, cinema and television, as well as video games and graphic novels.

An essential element of historical fiction is that it is set in the past and pays attention to the manners, social conditions and other details of the period depicted. Authors also frequently choose to explore notable historical figures in these settings, allowing readers to better understand how these individuals might have responded to their environments. Some subgenres such as alternate history and historical fantasy insert speculative or ahistorical elements into a novel.

Works of historical fiction are sometimes criticized for lack of authenticity because of readerly or genre expectations for accurate period details. This tension between historical authenticity, or historicity, and fiction frequently becomes a point of comment for readers and popular critics, while scholarly criticism frequently goes beyond this commentary, investigating the genre for its other thematic and critical interests.

Historical fiction, as a contemporary Western literary genre, has its foundations in the early 19th century works of Sir Walter Scott and his contemporaries in other national literatures such as the Frenchman Honoré de Balzac, the American James Fenimore Cooper, and a later Russian, Leo Tolstoy. However, the melding of "historical" and "fiction" in individual works of literature has a long tradition in most cultures; both western traditions (as early as Ancient Greek and Roman literature) as well as Eastern, in the form of oral and folk traditions (see mythology and folklore), which produced epics, novels, plays and other fictional works describing history for contemporary audiences.
""",
    'Western':
"""The Western is a genre of various arts which tell stories set primarily in the latter half of the 19th century in the American Old West, often centering on the life of a nomadic cowboy or gunfighter armed with a revolver and a rifle who rides a horse. Cowboys and gunslingers typically wear Stetson hats, bandannas, spurs, cowboy boots and buckskins. Recurring characters include the aforementioned cowboys, Native Americans, bandits, lawmen, bounty hunters, outlaws, soldiers (especially mounted cavalry, such as buffalo soldiers), and settlers (farmers, ranchers, and townsfolk). The ambience is usually punctuated with a Western music score, including American and Mexican folk music such as country, Native American music, New Mexico music, and rancheras.

Westerns often stress the harshness of the wilderness and frequently set the action in an arid, desolate landscape of deserts and mountains. Often, the vast landscape plays an important role, presenting a "...mythic vision of the plains and deserts of the American West". Specific settings include ranches, small frontier towns, saloons, railways and isolated military forts of the Wild West.
""",
    'Thriller':
"""Thriller is a broad genre of literature, film and television, having numerous, often overlapping subgenres. Thrillers are characterized and defined by the moods they elicit, giving viewers heightened feelings of suspense, excitement, surprise, anticipation and anxiety. Successful examples of thrillers are the films of Alfred Hitchcock.

Thrillers generally keep the audience on the "edge of their seats" as the plot builds towards a climax. The cover-up of important information is a common element. Literary devices such as red herrings, plot twists, and cliffhangers are used extensively. A thriller is usually a villain-driven plot, whereby he or she presents obstacles that the protagonist must overcome.

Homer's Odyssey is one of the oldest stories in the Western world and is regarded as an early prototype of the genre.
""",
    'Crime':
"""Crime cinema, in the broadest sense, is a cinematic genre inspired by and analogous to the crime fiction literary genre. Films of this genre generally involve various aspects of crime and its detection. Stylistically, the genre may overlap and combine with many other genres, such as drama or gangster film, but also include comedy, and, in turn, is divided into many sub-genres, such as mystery, suspense or noir.
""",
    'Science Fiction':
"""Science fiction film (or sci-fi film) is a genre that uses speculative, fictional science-based depictions of phenomena that are not fully accepted by mainstream science, such as extraterrestrial lifeforms, alien worlds, extrasensory perception and time travel, along with futuristic elements such as spacecraft, robots, cyborgs, interstellar travel or other technologies. Science fiction films have often been used to focus on political or social issues, and to explore philosophical issues like the human condition. In many cases, tropes derived from written science fiction may be used by filmmakers ignorant of or at best indifferent to the standards of scientific plausibility and plot logic to which written science fiction is traditionally held.

The genre has existed since the early years of silent cinema, when Georges Melies' A Trip to the Moon (1902) employed trick photography effects. The next major example in the genre was the film Metropolis (1927) - being the first feature length science fiction movie. From the 1930s to the 1950s, the genre consisted mainly of low-budget B movies. After Stanley Kubrick's landmark 2001: A Space Odyssey (1968), the science fiction film genre was taken more seriously. In the late 1970s, big-budget science fiction films filled with special effects became popular with audiences after the success of Star Wars and paved the way for the blockbuster hits of subsequent decades.
""",
    'Mystery':
"""A mystery film is a genre of film that revolves around the solution of a problem or a crime. It focuses on the efforts of the detective, private investigator or amateur sleuth to solve the mysterious circumstances of an issue by means of clues, investigation, and clever deduction.

The plot often centers on the deductive ability, prowess, confidence, or diligence of the detective as they attempt to unravel the crime or situation by piecing together clues and circumstances, seeking evidence, interrogating witnesses, and tracking down a criminal.

Suspense is often maintained as an important plot element. This can be done through the use of the soundtrack, camera angles, heavy shadows, and surprising plot twists. Alfred Hitchcock used all of these techniques, but would sometimes allow the audience in on a pending threat then draw out the moment for dramatic effect.

This genre has ranged from early mystery tales, fictional or literary detective stories, to classic Hitchcockian suspense-thrillers to classic private detective films. A related film subgenre is spy films.
""",
    'Documentary':
"""A documentary film is a nonfictional motion picture intended to document some aspect of reality, primarily for the purposes of instruction, education, or maintaining a historical record. Such films were originally shot on film stock—the only medium available—but now include video and digital productions that can be either direct-to-video, made into a TV show, or released for screening in cinemas. "Documentary" has been described as a "filmmaking practice, a cinematic tradition, and mode of audience reception" that is continually evolving and is without clear boundaries. Documentary films were originally called 'actuality' films and were only a minute or less in length. Over time documentaries have evolved to be longer in length and to include more categories, such as educational, observational, and even 'docufiction'. Documentaries are also educational and often used in schools to teach various principles. Social media platforms such as YouTube, have allowed documentary films to improve the ways the films are distributed and able to educate and broaden the reach of people who receive the information.
""",
})

In [60]:
genres = []

for name in descriptions:
    genres.append({'name': name, 'description': descriptions[name]})

In [61]:
for genre in genres:
    name = genre['name']
    file = open('./genres/{}.jpg'.format(name), 'rb').read()
    res = base64.b64encode(file)
    genre['image'] = str(res)

In [62]:
frame = pd.DataFrame(genres)
frame.head()

,description,image,name
0,Musical film is a film genre in which songs su...,b'/9j/4AAQSkZJRgABAgAAAQABAAD/2wBDAAgGBgcGBQgH...,Music
1,Romance films or romance movies are romantic l...,b'/9j/4AAQSkZJRgABAgAAZABkAAD/4QAwRXhpZgAATU0A...,Romance
2,"A children's film, or family film, is a film g...",b'/9j/4AAQSkZJRgABAQEBLAEsAAD/2wBDAAUDBAQEAwUE...,Family
3,War film is a film genre concerned with warfar...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEhUT...,War
4,Adventure films are a genre of film that typic...,b'/9j/4AAQSkZJRgABAQAAAQABAAD//gA7Q1JFQVRPUjog...,Adventure


In [21]:
g_id = 35

file = open('./genres/Crime.jpg', 'rb').read()
image = str(base64.b64encode(file))[2:-1]
res = requests.get(base_url+'genre/{}'.format(g_id))
genre = json.loads(res.text)
genre['image'] = image
requests.put(url=base_url+'genre/{}/'.format(g_id), data=json.dumps(genre))

<Response [200]>

# Post genres

In [63]:
def post_genre(data):
    url = base_url + 'genre'
    res = requests.post(url=url, data=json.dumps(data))
    return res

In [65]:
for i, genre in enumerate(genres):
    post_genre(genre)
    print('Posted {} of {}'.format(i+1, len(genres)))

Posted 1 of 18
Posted 2 of 18
Posted 3 of 18
Posted 4 of 18
Posted 5 of 18
Posted 6 of 18
Posted 7 of 18
Posted 8 of 18
Posted 9 of 18
Posted 10 of 18
Posted 11 of 18
Posted 12 of 18
Posted 13 of 18
Posted 14 of 18
Posted 15 of 18
Posted 16 of 18
Posted 17 of 18
Posted 18 of 18


# Create companies

In [98]:
data = pd.read_csv('movies_metadata.csv')
data = data.loc[np.arange(0, 500)]

C:\Users\stepan.samsonov\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [99]:
names = set()

for line in data['production_companies'].iteritems():
    line = eval(line[1])
    print(line)
    for elem in line:
        names.add(elem['name'])

[{'name': 'Pixar Animation Studios', 'id': 3}]
[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]
[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]
[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]
[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]
[{'name': 'Regency Enterprises', 'id': 508}, {'name': 'Forward Pass', 'id': 675}, {'name': 'Warner Bros.', 'id': 6194}]
[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Scott Rudin Productions', 'id': 258}, {'name': 'Mirage Enterprises', 'id': 932}, {'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Constellation Entertainment', 'id': 14941}, {'name': 'Worldwide', 'id': 55873}, {'name': 'Mont Blanc Entertainment GmbH', 'id': 58079}]
[{'name': 'Walt Disney Pictures', 'id': 2}]
[{'name': 'Universal Pictures', 'id': 33}, {'name': 'Imperial Entertainment', 'id': 21437}, {'

In [100]:
dates = dict()
genres = dict()

for line in data.iterrows():
    line = line[1]
    companies = eval(line['production_companies'])
    film_genres = eval(line['genres'])
    print(companies)
    for company in companies:
        name = company['name']
        if dates.get(name):
            dates[name].append(line['release_date'])
        else:
            dates[name] = [line['release_date']]
            
        for genre in film_genres:
            if genres.get(name):
                genres[name].add(genre['name'])
            else:
                genres[name] = set()
                genres[name].add(genre['name'])

[{'name': 'Pixar Animation Studios', 'id': 3}]
[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]
[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]
[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]
[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]
[{'name': 'Regency Enterprises', 'id': 508}, {'name': 'Forward Pass', 'id': 675}, {'name': 'Warner Bros.', 'id': 6194}]
[{'name': 'Paramount Pictures', 'id': 4}, {'name': 'Scott Rudin Productions', 'id': 258}, {'name': 'Mirage Enterprises', 'id': 932}, {'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Constellation Entertainment', 'id': 14941}, {'name': 'Worldwide', 'id': 55873}, {'name': 'Mont Blanc Entertainment GmbH', 'id': 58079}]
[{'name': 'Walt Disney Pictures', 'id': 2}]
[{'name': 'Universal Pictures', 'id': 33}, {'name': 'Imperial Entertainment', 'id': 21437}, {'

In [101]:
companies = []

for company in dates:
    years = []
    company_genres = []
    for date in dates[company]:
        if pd.isnull(date):
            pass
        else:
            date = date.split('-')[0]
            years.append(int(date))    
            
    if genres.get(company):
        company_genres = list(genres[company])
        
    companies.append({'name': company, 'founding_date': min(years), 'genres': company_genres})

In [102]:
frame = pd.DataFrame(companies)
frame.head()

,founding_date,genres,name
0,1995,"[Comedy, Family, Animation]",Pixar Animation Studios
1,1993,"[Romance, Action, Thriller, War, Adventure, Co...",TriStar Pictures
2,1995,"[Adventure, Family, Fantasy]",Teitler Film
3,1994,"[Thriller, Adventure, Music, Comedy, Family, D...",Interscope Communications
4,1993,"[Romance, Action, Thriller, War, Crime, Advent...",Warner Bros.


# Post companies

In [145]:
def post_company(data):
    url = base_url + 'company'
    res = requests.post(url=url, data=json.dumps(data))
    return res

def names_to_id(names, data):
    res = []
    
    for name in names:
        elem = list(filter(lambda x: x['name'] == name, data))
        
        if len(elem):
            elem = elem[0]
        else:
            continue
        
        res.append(elem['id'])
        
    return res

In [104]:
res = requests.get(url=base_url+'genre')
genres = json.loads(res.text)

for i, company in enumerate(companies):
    company['genres'] = names_to_id(company['genres'], genres)
    post_company(company)
    print('Posted {} of {}'.format(i+1, len(companies)))

Posted 1 of 510
Posted 2 of 510
Posted 3 of 510
Posted 4 of 510
Posted 5 of 510
Posted 6 of 510
Posted 7 of 510
Posted 8 of 510
Posted 9 of 510
Posted 10 of 510
Posted 11 of 510
Posted 12 of 510
Posted 13 of 510
Posted 14 of 510
Posted 15 of 510
Posted 16 of 510
Posted 17 of 510
Posted 18 of 510
Posted 19 of 510
Posted 20 of 510
Posted 21 of 510
Posted 22 of 510
Posted 23 of 510
Posted 24 of 510
Posted 25 of 510
Posted 26 of 510
Posted 27 of 510
Posted 28 of 510
Posted 29 of 510
Posted 30 of 510
Posted 31 of 510
Posted 32 of 510
Posted 33 of 510
Posted 34 of 510
Posted 35 of 510
Posted 36 of 510
Posted 37 of 510
Posted 38 of 510
Posted 39 of 510
Posted 40 of 510
Posted 41 of 510
Posted 42 of 510
Posted 43 of 510
Posted 44 of 510
Posted 45 of 510
Posted 46 of 510
Posted 47 of 510
Posted 48 of 510
Posted 49 of 510
Posted 50 of 510
Posted 51 of 510
Posted 52 of 510
Posted 53 of 510
Posted 54 of 510
Posted 55 of 510
Posted 56 of 510
Posted 57 of 510
Posted 58 of 510
Posted 59 of 510
Posted

Posted 463 of 510
Posted 464 of 510
Posted 465 of 510
Posted 466 of 510
Posted 467 of 510
Posted 468 of 510
Posted 469 of 510
Posted 470 of 510
Posted 471 of 510
Posted 472 of 510
Posted 473 of 510
Posted 474 of 510
Posted 475 of 510
Posted 476 of 510
Posted 477 of 510
Posted 478 of 510
Posted 479 of 510
Posted 480 of 510
Posted 481 of 510
Posted 482 of 510
Posted 483 of 510
Posted 484 of 510
Posted 485 of 510
Posted 486 of 510
Posted 487 of 510
Posted 488 of 510
Posted 489 of 510
Posted 490 of 510
Posted 491 of 510
Posted 492 of 510
Posted 493 of 510
Posted 494 of 510
Posted 495 of 510
Posted 496 of 510
Posted 497 of 510
Posted 498 of 510
Posted 499 of 510
Posted 500 of 510
Posted 501 of 510
Posted 502 of 510
Posted 503 of 510
Posted 504 of 510
Posted 505 of 510
Posted 506 of 510
Posted 507 of 510
Posted 508 of 510
Posted 509 of 510
Posted 510 of 510


# Create actors

In [122]:
data_credits = pd.read_csv('credits.csv')
data = []
data_film = pd.read_csv('movies_metadata.csv')
data_film = data_film.loc[np.arange(0, 500)]

C:\Users\stepan.samsonov\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [123]:
genres = dict()

for film in data_film.iterrows():
    film = film[1]
    film_genres = list(map(lambda x: x['name'], eval(film['genres'])))
    genres[film['id']] = film_genres

In [124]:
for row in data_credits.iterrows():
    if str(row[1]['id']) in film_ids:
        data.append(row[1])

In [125]:
data = pd.DataFrame(data)
data.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [136]:
actor_data = dict()

for line in data.iterrows():
    line = line[1]
    cast = eval(line['cast'])
    film_genres = genres[str(line['id'])]
    for elem in cast:
        actor_name = elem['name']
        if actor_data.get(actor_name):
            for genre in film_genres:
                actor_data[actor_name]['genres'].add(genre)
        else:
            actor_data[actor_name] = {
                'image': elem['profile_path'],
                'genres': set(),
            }
            for genre in film_genres:
                actor_data[actor_name]['genres'].add(genre)

In [139]:
actors = []

for actor_name in actor_data:
    actor = actor_data[actor_name]
    actors.append({'name': actor_name, 'image': actor['image'], 'genres': list(actor['genres'])})

In [140]:
frame = pd.DataFrame(actors)
frame.head()

,genres,image,name
0,"[Romance, Comedy, Family, Drama, Animation]",/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,Tom Hanks
1,"[Comedy, Family, Drama, Animation, Fantasy]",/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,Tim Allen
2,"[Crime, Comedy, Family, Drama, Animation]",/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,Don Rickles
3,"[Comedy, Family, Animation]",/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,Jim Varney
4,"[Romance, Action, Adventure, Comedy, Family, D...",/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,Wallace Shawn


# Post actors

In [142]:
def post_actor(data):
    url = base_url + 'actor'
    res = requests.post(url=url, data=json.dumps(data))
    return res

In [144]:
res = requests.get(url=base_url+'genre')
genres = json.loads(res.text)

for i, actor in enumerate(actors):
    actor['genres'] = names_to_id(actor['genres'], genres)
    post_actor(actor)
    print('Posted {} of {}'.format(i+1, len(actors)))

Posted 1 of 6241
Posted 2 of 6241
Posted 3 of 6241
Posted 4 of 6241
Posted 5 of 6241
Posted 6 of 6241
Posted 7 of 6241
Posted 8 of 6241
Posted 9 of 6241
Posted 10 of 6241
Posted 11 of 6241
Posted 12 of 6241
Posted 13 of 6241
Posted 14 of 6241
Posted 15 of 6241
Posted 16 of 6241
Posted 17 of 6241
Posted 18 of 6241
Posted 19 of 6241
Posted 20 of 6241
Posted 21 of 6241
Posted 22 of 6241
Posted 23 of 6241
Posted 24 of 6241
Posted 25 of 6241
Posted 26 of 6241
Posted 27 of 6241
Posted 28 of 6241
Posted 29 of 6241
Posted 30 of 6241
Posted 31 of 6241
Posted 32 of 6241
Posted 33 of 6241
Posted 34 of 6241
Posted 35 of 6241
Posted 36 of 6241
Posted 37 of 6241
Posted 38 of 6241
Posted 39 of 6241
Posted 40 of 6241
Posted 41 of 6241
Posted 42 of 6241
Posted 43 of 6241
Posted 44 of 6241
Posted 45 of 6241
Posted 46 of 6241
Posted 47 of 6241
Posted 48 of 6241
Posted 49 of 6241
Posted 50 of 6241
Posted 51 of 6241
Posted 52 of 6241
Posted 53 of 6241
Posted 54 of 6241
Posted 55 of 6241
Posted 56 of 6241
P

Posted 439 of 6241
Posted 440 of 6241
Posted 441 of 6241
Posted 442 of 6241
Posted 443 of 6241
Posted 444 of 6241
Posted 445 of 6241
Posted 446 of 6241
Posted 447 of 6241
Posted 448 of 6241
Posted 449 of 6241
Posted 450 of 6241
Posted 451 of 6241
Posted 452 of 6241
Posted 453 of 6241
Posted 454 of 6241
Posted 455 of 6241
Posted 456 of 6241
Posted 457 of 6241
Posted 458 of 6241
Posted 459 of 6241
Posted 460 of 6241
Posted 461 of 6241
Posted 462 of 6241
Posted 463 of 6241
Posted 464 of 6241
Posted 465 of 6241
Posted 466 of 6241
Posted 467 of 6241
Posted 468 of 6241
Posted 469 of 6241
Posted 470 of 6241
Posted 471 of 6241
Posted 472 of 6241
Posted 473 of 6241
Posted 474 of 6241
Posted 475 of 6241
Posted 476 of 6241
Posted 477 of 6241
Posted 478 of 6241
Posted 479 of 6241
Posted 480 of 6241
Posted 481 of 6241
Posted 482 of 6241
Posted 483 of 6241
Posted 484 of 6241
Posted 485 of 6241
Posted 486 of 6241
Posted 487 of 6241
Posted 488 of 6241
Posted 489 of 6241
Posted 490 of 6241
Posted 491 o

Posted 873 of 6241
Posted 874 of 6241
Posted 875 of 6241
Posted 876 of 6241
Posted 877 of 6241
Posted 878 of 6241
Posted 879 of 6241
Posted 880 of 6241
Posted 881 of 6241
Posted 882 of 6241
Posted 883 of 6241
Posted 884 of 6241
Posted 885 of 6241
Posted 886 of 6241
Posted 887 of 6241
Posted 888 of 6241
Posted 889 of 6241
Posted 890 of 6241
Posted 891 of 6241
Posted 892 of 6241
Posted 893 of 6241
Posted 894 of 6241
Posted 895 of 6241
Posted 896 of 6241
Posted 897 of 6241
Posted 898 of 6241
Posted 899 of 6241
Posted 900 of 6241
Posted 901 of 6241
Posted 902 of 6241
Posted 903 of 6241
Posted 904 of 6241
Posted 905 of 6241
Posted 906 of 6241
Posted 907 of 6241
Posted 908 of 6241
Posted 909 of 6241
Posted 910 of 6241
Posted 911 of 6241
Posted 912 of 6241
Posted 913 of 6241
Posted 914 of 6241
Posted 915 of 6241
Posted 916 of 6241
Posted 917 of 6241
Posted 918 of 6241
Posted 919 of 6241
Posted 920 of 6241
Posted 921 of 6241
Posted 922 of 6241
Posted 923 of 6241
Posted 924 of 6241
Posted 925 o

Posted 1291 of 6241
Posted 1292 of 6241
Posted 1293 of 6241
Posted 1294 of 6241
Posted 1295 of 6241
Posted 1296 of 6241
Posted 1297 of 6241
Posted 1298 of 6241
Posted 1299 of 6241
Posted 1300 of 6241
Posted 1301 of 6241
Posted 1302 of 6241
Posted 1303 of 6241
Posted 1304 of 6241
Posted 1305 of 6241
Posted 1306 of 6241
Posted 1307 of 6241
Posted 1308 of 6241
Posted 1309 of 6241
Posted 1310 of 6241
Posted 1311 of 6241
Posted 1312 of 6241
Posted 1313 of 6241
Posted 1314 of 6241
Posted 1315 of 6241
Posted 1316 of 6241
Posted 1317 of 6241
Posted 1318 of 6241
Posted 1319 of 6241
Posted 1320 of 6241
Posted 1321 of 6241
Posted 1322 of 6241
Posted 1323 of 6241
Posted 1324 of 6241
Posted 1325 of 6241
Posted 1326 of 6241
Posted 1327 of 6241
Posted 1328 of 6241
Posted 1329 of 6241
Posted 1330 of 6241
Posted 1331 of 6241
Posted 1332 of 6241
Posted 1333 of 6241
Posted 1334 of 6241
Posted 1335 of 6241
Posted 1336 of 6241
Posted 1337 of 6241
Posted 1338 of 6241
Posted 1339 of 6241
Posted 1340 of 6241


Posted 1703 of 6241
Posted 1704 of 6241
Posted 1705 of 6241
Posted 1706 of 6241
Posted 1707 of 6241
Posted 1708 of 6241
Posted 1709 of 6241
Posted 1710 of 6241
Posted 1711 of 6241
Posted 1712 of 6241
Posted 1713 of 6241
Posted 1714 of 6241
Posted 1715 of 6241
Posted 1716 of 6241
Posted 1717 of 6241
Posted 1718 of 6241
Posted 1719 of 6241
Posted 1720 of 6241
Posted 1721 of 6241
Posted 1722 of 6241
Posted 1723 of 6241
Posted 1724 of 6241
Posted 1725 of 6241
Posted 1726 of 6241
Posted 1727 of 6241
Posted 1728 of 6241
Posted 1729 of 6241
Posted 1730 of 6241
Posted 1731 of 6241
Posted 1732 of 6241
Posted 1733 of 6241
Posted 1734 of 6241
Posted 1735 of 6241
Posted 1736 of 6241
Posted 1737 of 6241
Posted 1738 of 6241
Posted 1739 of 6241
Posted 1740 of 6241
Posted 1741 of 6241
Posted 1742 of 6241
Posted 1743 of 6241
Posted 1744 of 6241
Posted 1745 of 6241
Posted 1746 of 6241
Posted 1747 of 6241
Posted 1748 of 6241
Posted 1749 of 6241
Posted 1750 of 6241
Posted 1751 of 6241
Posted 1752 of 6241


...

Posted 4583 of 6241
Posted 4584 of 6241
Posted 4585 of 6241
Posted 4586 of 6241
Posted 4587 of 6241
Posted 4588 of 6241
Posted 4589 of 6241
Posted 4590 of 6241
Posted 4591 of 6241
Posted 4592 of 6241
Posted 4593 of 6241
Posted 4594 of 6241
Posted 4595 of 6241
Posted 4596 of 6241
Posted 4597 of 6241
Posted 4598 of 6241
Posted 4599 of 6241
Posted 4600 of 6241
Posted 4601 of 6241
Posted 4602 of 6241
Posted 4603 of 6241
Posted 4604 of 6241
Posted 4605 of 6241
Posted 4606 of 6241
Posted 4607 of 6241
Posted 4608 of 6241
Posted 4609 of 6241
Posted 4610 of 6241
Posted 4611 of 6241
Posted 4612 of 6241
Posted 4613 of 6241
Posted 4614 of 6241
Posted 4615 of 6241
Posted 4616 of 6241
Posted 4617 of 6241
Posted 4618 of 6241
Posted 4619 of 6241
Posted 4620 of 6241
Posted 4621 of 6241
Posted 4622 of 6241
Posted 4623 of 6241
Posted 4624 of 6241
Posted 4625 of 6241
Posted 4626 of 6241
Posted 4627 of 6241
Posted 4628 of 6241
Posted 4629 of 6241
Posted 4630 of 6241
Posted 4631 of 6241
Posted 4632 of 6241


Posted 4995 of 6241
Posted 4996 of 6241
Posted 4997 of 6241
Posted 4998 of 6241
Posted 4999 of 6241
Posted 5000 of 6241
Posted 5001 of 6241
Posted 5002 of 6241
Posted 5003 of 6241
Posted 5004 of 6241
Posted 5005 of 6241
Posted 5006 of 6241
Posted 5007 of 6241
Posted 5008 of 6241
Posted 5009 of 6241
Posted 5010 of 6241
Posted 5011 of 6241
Posted 5012 of 6241
Posted 5013 of 6241
Posted 5014 of 6241
Posted 5015 of 6241
Posted 5016 of 6241
Posted 5017 of 6241
Posted 5018 of 6241
Posted 5019 of 6241
Posted 5020 of 6241
Posted 5021 of 6241
Posted 5022 of 6241
Posted 5023 of 6241
Posted 5024 of 6241
Posted 5025 of 6241
Posted 5026 of 6241
Posted 5027 of 6241
Posted 5028 of 6241
Posted 5029 of 6241
Posted 5030 of 6241
Posted 5031 of 6241
Posted 5032 of 6241
Posted 5033 of 6241
Posted 5034 of 6241
Posted 5035 of 6241
Posted 5036 of 6241
Posted 5037 of 6241
Posted 5038 of 6241
Posted 5039 of 6241
Posted 5040 of 6241
Posted 5041 of 6241
Posted 5042 of 6241
Posted 5043 of 6241
Posted 5044 of 6241


Posted 5408 of 6241
Posted 5409 of 6241
Posted 5410 of 6241
Posted 5411 of 6241
Posted 5412 of 6241
Posted 5413 of 6241
Posted 5414 of 6241
Posted 5415 of 6241
Posted 5416 of 6241
Posted 5417 of 6241
Posted 5418 of 6241
Posted 5419 of 6241
Posted 5420 of 6241
Posted 5421 of 6241
Posted 5422 of 6241
Posted 5423 of 6241
Posted 5424 of 6241
Posted 5425 of 6241
Posted 5426 of 6241
Posted 5427 of 6241
Posted 5428 of 6241
Posted 5429 of 6241
Posted 5430 of 6241
Posted 5431 of 6241
Posted 5432 of 6241
Posted 5433 of 6241
Posted 5434 of 6241
Posted 5435 of 6241
Posted 5436 of 6241
Posted 5437 of 6241
Posted 5438 of 6241
Posted 5439 of 6241
Posted 5440 of 6241
Posted 5441 of 6241
Posted 5442 of 6241
Posted 5443 of 6241
Posted 5444 of 6241
Posted 5445 of 6241
Posted 5446 of 6241
Posted 5447 of 6241
Posted 5448 of 6241
Posted 5449 of 6241
Posted 5450 of 6241
Posted 5451 of 6241
Posted 5452 of 6241
Posted 5453 of 6241
Posted 5454 of 6241
Posted 5455 of 6241
Posted 5456 of 6241
Posted 5457 of 6241


Posted 5821 of 6241
Posted 5822 of 6241
Posted 5823 of 6241
Posted 5824 of 6241
Posted 5825 of 6241
Posted 5826 of 6241
Posted 5827 of 6241
Posted 5828 of 6241
Posted 5829 of 6241
Posted 5830 of 6241
Posted 5831 of 6241
Posted 5832 of 6241
Posted 5833 of 6241
Posted 5834 of 6241
Posted 5835 of 6241
Posted 5836 of 6241
Posted 5837 of 6241
Posted 5838 of 6241
Posted 5839 of 6241
Posted 5840 of 6241
Posted 5841 of 6241
Posted 5842 of 6241
Posted 5843 of 6241
Posted 5844 of 6241
Posted 5845 of 6241
Posted 5846 of 6241
Posted 5847 of 6241
Posted 5848 of 6241
Posted 5849 of 6241
Posted 5850 of 6241
Posted 5851 of 6241
Posted 5852 of 6241
Posted 5853 of 6241
Posted 5854 of 6241
Posted 5855 of 6241
Posted 5856 of 6241
Posted 5857 of 6241
Posted 5858 of 6241
Posted 5859 of 6241
Posted 5860 of 6241
Posted 5861 of 6241
Posted 5862 of 6241
Posted 5863 of 6241
Posted 5864 of 6241
Posted 5865 of 6241
Posted 5866 of 6241
Posted 5867 of 6241
Posted 5868 of 6241
Posted 5869 of 6241
Posted 5870 of 6241


Posted 6234 of 6241
Posted 6235 of 6241
Posted 6236 of 6241
Posted 6237 of 6241
Posted 6238 of 6241
Posted 6239 of 6241
Posted 6240 of 6241
Posted 6241 of 6241


# Create films

In [32]:
data = pd.read_csv('movies_metadata.csv')
data = data.loc[np.arange(0, 500)]

C:\Users\stepan.samsonov\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [149]:
films = []
i = 0

for line in data.iterrows():
    line = line[1]
    
    countries = eval(line['production_countries'])
    y, m, d = line['release_date'].split('-')
    date = '/'.join([d, m, y])
    
    film = {
        'budget': line['budget'],
        'name': line['title'],
        'language': line['original_language'],
        'overview': line['overview'],
        'country': countries[0]['name'] if len(countries) else None,
        'date': date,
        'revenue': line['revenue'],
        'runtime': line['runtime'],
        'tagline': line['tagline'],
        'image': line['poster_path']
        'actors': list(map(lambda x: x['name'], eval(data_credits[data_credits['id'] == int(line['id'])]['cast'][i]))),
        'genres': list(map(lambda x: x['name'], eval(line['genres']))),
        'companies': list(map(lambda x: x['name'], eval(line['production_companies']))),
    }
    films.append(film)
    i += 1

In [150]:
frame = pd.DataFrame(films)
frame.head()

,actors,budget,companies,country,date,genres,language,name,overview,revenue,runtime,tagline
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",30000000,[Pixar Animation Studios],United States of America,30/10/1995,"[Animation, Comedy, Family]",en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",373554033.0,81.0,NaN
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",65000000,"[TriStar Pictures, Teitler Film, Interscope Co...",United States of America,15/12/1995,"[Adventure, Fantasy, Family]",en,Jumanji,When siblings Judy and Peter discover an encha...,262797249.0,104.0,Roll the dice and unleash the excitement!
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",0,"[Warner Bros., Lancaster Gate]",United States of America,22/12/1995,"[Romance, Comedy]",en,Grumpier Old Men,A family wedding reignites the ancient feud be...,0.0,101.0,Still Yelling. Still Fighting. Still Ready for...
3,"[Whitney Houston, Angela Bassett, Loretta Devi...",16000000,[Twentieth Century Fox Film Corporation],United States of America,22/12/1995,"[Comedy, Drama, Romance]",en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",81452156.0,127.0,Friends are the people who let you be yourself...
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...",0,"[Sandollar Productions, Touchstone Pictures]",United States of America,10/02/1995,[Comedy],en,Father of the Bride Part II,Just when George Banks has recovered from his ...,76578911.0,106.0,Just When His World Is Back To Normal... He's ...


# Post films

In [156]:
def post_film(data):
    url = base_url + 'film'
    res = requests.post(url=url, data=json.dumps(data))
    return res

In [157]:
genres = requests.get(url=base_url+'genre')
genres = json.loads(genres.text)

actors = requests.get(url=base_url+'actor')
actors = json.loads(actors.text)

companies = requests.get(url=base_url+'company')
companies = json.loads(companies.text)

for i, film in enumerate(films):
    film['genres'] = names_to_id(film['genres'], genres)
    film['actors'] = names_to_id(film['actors'], actors)
    film['companies'] = names_to_id(film['companies'], companies)
    post_film(film)
    print('Posted {} of {}'.format(i+1, len(films)))

Posted 1 of 500
Posted 2 of 500
Posted 3 of 500
Posted 4 of 500
Posted 5 of 500
Posted 6 of 500
Posted 7 of 500
Posted 8 of 500
Posted 9 of 500
Posted 10 of 500
Posted 11 of 500
Posted 12 of 500
Posted 13 of 500
Posted 14 of 500
Posted 15 of 500
Posted 16 of 500
Posted 17 of 500
Posted 18 of 500
Posted 19 of 500
Posted 20 of 500
Posted 21 of 500
Posted 22 of 500
Posted 23 of 500
Posted 24 of 500
Posted 25 of 500
Posted 26 of 500
Posted 27 of 500
Posted 28 of 500
Posted 29 of 500
Posted 30 of 500
Posted 31 of 500
Posted 32 of 500
Posted 33 of 500
Posted 34 of 500
Posted 35 of 500
Posted 36 of 500
Posted 37 of 500
Posted 38 of 500
Posted 39 of 500
Posted 40 of 500
Posted 41 of 500
Posted 42 of 500
Posted 43 of 500
Posted 44 of 500
Posted 45 of 500
Posted 46 of 500
Posted 47 of 500
Posted 48 of 500
Posted 49 of 500
Posted 50 of 500
Posted 51 of 500
Posted 52 of 500
Posted 53 of 500
Posted 54 of 500
Posted 55 of 500
Posted 56 of 500
Posted 57 of 500
Posted 58 of 500
Posted 59 of 500
Posted

Posted 463 of 500
Posted 464 of 500
Posted 465 of 500
Posted 466 of 500
Posted 467 of 500
Posted 468 of 500
Posted 469 of 500
Posted 470 of 500
Posted 471 of 500
Posted 472 of 500
Posted 473 of 500
Posted 474 of 500
Posted 475 of 500
Posted 476 of 500
Posted 477 of 500
Posted 478 of 500
Posted 479 of 500
Posted 480 of 500
Posted 481 of 500
Posted 482 of 500
Posted 483 of 500
Posted 484 of 500
Posted 485 of 500
Posted 486 of 500
Posted 487 of 500
Posted 488 of 500
Posted 489 of 500
Posted 490 of 500
Posted 491 of 500
Posted 492 of 500
Posted 493 of 500
Posted 494 of 500
Posted 495 of 500
Posted 496 of 500
Posted 497 of 500
Posted 498 of 500
Posted 499 of 500
Posted 500 of 500


# Create reviews

In [174]:
data = pd.read_csv('movies_metadata.csv')
data = data.loc[np.arange(0, 500)]

C:\Users\stepan.samsonov\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [175]:
def get_local_id(imdb_id, mode, opinion):
    path = 'aclImdb//{}//urls_{}.txt'.format(mode, opinion)
    lines = open(path).readlines()
    
    for i, line in enumerate(lines):
        if line.find(imdb_id) != -1:
            return i
    return None

def get_reviews(local_id, mode, opinion):
    list_dir = glob.glob('aclImdb\\{}\\{}\\*.txt'.format(mode, opinion))
    res = []
    
    for path in list_dir:
        file_name = path.split('\\')[-1]
        elem_id, _ = file_name.split('_')
        if int(elem_id) == local_id:
            text = open(path).read()
            res.append(text)
    return res

In [176]:
modes = ['train', 'test']
opinions = ['pos', 'neg']
reviews_data = dict()

for film in data.iterrows():
    film = film[1]
    imdb_id = film['imdb_id']
    res = []
    
    for mode in modes:
        for opinion in opinions:
            local_id = get_local_id(imdb_id, mode, opinion)
            
            if local_id:
                new_reviews = get_reviews(local_id, mode, opinion)
                review_instaces = []
                for review in new_reviews:
                    review_instaces.append({
                        'is_positive': opinion == 'pos',
                        'text': review,
                    })
                res += review_instaces
                
    reviews_data[film['title']] = res

In [177]:
reviews = []

for film_name in reviews_data:
    film_reviews = reviews_data[film_name]
    for review in film_reviews:
        review['film'] = film_name
        reviews.append(review)

In [178]:
frame = pd.DataFrame(reviews)
frame.head()

,film,is_positive,text
0,Sudden Death,False,"Okay, here is a really short review: this movi..."
1,The American President,False,I rented this movie the other night because ne...
2,Sense and Sensibility,False,When I first saw this film in cinema 11 years ...
3,Leaving Las Vegas,False,Cage plays a drunk and gets high critically pr...
4,Othello,True,"Yes, Shakespeare would indeed have been proud...."


# Post reviews

In [179]:
def post_review(data):
    url = base_url + 'review'
    res = requests.post(url=url, data=json.dumps(data))
    return res

def id_film_by_name(name, films):
    for film in films:
        if film['name'] == name:
            return film['id']
    return None

In [180]:
films = requests.get(base_url+'film')
films = json.loads(films.text)

for i, review in enumerate(reviews):
    film_id = id_film_by_name(review['film'], films)
    
    if film_id:
        review['film'] = film_id
    else:
        print('ERROR', i)
        break
        
    post_review(review)    
    print('Posted {} of {}'.format(i+1, len(reviews)))

Posted 1 of 106
Posted 2 of 106
Posted 3 of 106
Posted 4 of 106
Posted 5 of 106
Posted 6 of 106
Posted 7 of 106
Posted 8 of 106
Posted 9 of 106
Posted 10 of 106
Posted 11 of 106
Posted 12 of 106
Posted 13 of 106
Posted 14 of 106
Posted 15 of 106
Posted 16 of 106
Posted 17 of 106
Posted 18 of 106
Posted 19 of 106
Posted 20 of 106
Posted 21 of 106
Posted 22 of 106
Posted 23 of 106
Posted 24 of 106
Posted 25 of 106
Posted 26 of 106
Posted 27 of 106
Posted 28 of 106
Posted 29 of 106
Posted 30 of 106
Posted 31 of 106
Posted 32 of 106
Posted 33 of 106
Posted 34 of 106
Posted 35 of 106
Posted 36 of 106
Posted 37 of 106
Posted 38 of 106
Posted 39 of 106
Posted 40 of 106
Posted 41 of 106
Posted 42 of 106
Posted 43 of 106
Posted 44 of 106
Posted 45 of 106
Posted 46 of 106
Posted 47 of 106
Posted 48 of 106
Posted 49 of 106
Posted 50 of 106
Posted 51 of 106
Posted 52 of 106
Posted 53 of 106
Posted 54 of 106
Posted 55 of 106
Posted 56 of 106
Posted 57 of 106
Posted 58 of 106
Posted 59 of 106
Posted